In [1]:
# Two Naive Bayes classification algorithms
# 1. Discrete Naive Bayes: for categorical features(predictors)
# 2. Kernel Naive Bayes: For numerical Features(predictors)

In [3]:
# In this algorithm, the probability of a record according to a certain class is evaluated on the basics 


In [5]:
# The binning is implemented through pd.cut() function - for numeric to categorical conversion


In [103]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.metrics import classification_report, roc_auc_score

# BernoulliNB

In [67]:
df = pd.read_csv("Cancer.csv")
df

,subjid,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,1,40-49,premeno,15 to 19,0 to 2,yes,three,right,left_up,no,recurrence-events
1,2,50-59,ge40,15 to 19,0 to 2,no,one,right,central,no,no-recurrence-events
2,3,50-59,ge40,35 to 39,0 to 2,no,two,left,left_low,no,recurrence-events
3,4,40-49,premeno,35 to 39,0 to 2,yes,three,right,left_low,yes,no-recurrence-events
4,5,40-49,premeno,30 to 34,3 to 5,yes,two,left,right_up,no,recurrence-events
...,...,...,...,...,...,...,...,...,...,...,...
281,282,50-59,ge40,30 to 34,6 to 8,yes,two,left,left_low,no,no-recurrence-events
282,283,50-59,premeno,25 to 29,3 to 5,yes,two,left,left_low,yes,no-recurrence-events
283,284,30-39,premeno,30 to 34,6 to 8,yes,two,right,right_up,no,no-recurrence-events
284,285,50-59,premeno,15 to 19,0 to 2,no,two,right,left_low,no,no-recurrence-events


In [69]:
x = df.drop('Class', axis = 'columns')
y = df['Class']




In [71]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                   random_state = 24,
                                                   test_size = 0.3,
                                                   stratify = y)

In [79]:
ohe = OneHotEncoder(
    sparse_output = False,
    handle_unknown = "ignore",
).set_output(transform = "pandas")

nb = BernoulliNB()

pipe = Pipeline([("OHE", ohe), ("NB", nb)])

params = {"NB__alpha" : np.linspace(0.001, 3, 10)}

kfold = StratifiedKFold(n_splits = 5,
                        shuffle = True,
                        random_state = 24)

gcv = GridSearchCV(pipe, 
                  param_grid = params,
                  scoring = 'roc_auc',
                  cv = kfold)

gcv.fit(x, y)



GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('OHE',
                                        OneHotEncoder(handle_unknown='ignore',
                                                      sparse_output=False)),
                                       ('NB', BernoulliNB())]),
             param_grid={'NB__alpha': array([1.00000000e-03, 3.34222222e-01, 6.67444444e-01, 1.00066667e+00,
       1.33388889e+00, 1.66711111e+00, 2.00033333e+00, 2.33355556e+00,
       2.66677778e+00, 3.00000000e+00])},
             scoring='roc_auc')

In [81]:
y_pred = gcv.predict(x_test)

In [83]:
print(gcv.best_score_)

0.7194010043041608


In [61]:
print(classification_report(y_test, y_pred))

                      precision    recall  f1-score   support

no-recurrence-events       0.81      0.85      0.83        60
   recurrence-events       0.61      0.54      0.57        26

            accuracy                           0.76        86
           macro avg       0.71      0.69      0.70        86
        weighted avg       0.75      0.76      0.75        86



In [43]:
y_pred_prob = pipe.predict_proba(x_test)[:, 1]
print(roc_auc_score(y_test, y_pred_prob))

0.75


In [45]:
vars(BernoulliNB())

{'alpha': 1.0,
 'fit_prior': True,
 'class_prior': None,
 'force_alpha': True,
 'binarize': 0.0}

# GaussianNB

In [105]:
df = pd.read_csv("HR_comma_sep.csv")
df

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14990,0.40,0.57,2,151,3,0,1,0,support,low
14991,0.37,0.48,2,160,3,0,1,0,support,low
14992,0.37,0.53,2,143,3,0,1,0,support,low
14993,0.11,0.96,6,280,4,0,1,0,support,low


In [107]:
x = df.drop('left', axis = 'columns')
y = df['left']


In [91]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                   random_state = 24,
                                                   test_size = 0.3,
                                                   stratify = y)

In [109]:
ohe = OneHotEncoder(
    sparse_output = False,
    handle_unknown = "ignore",
).set_output(transform = "pandas")

nb = GaussianNB()

pipe = Pipeline([("OHE", ohe), ("NB", nb)])

params = {"NB__var_smoothing" : np.linspace(0.001, 3, 10)}

kfold = StratifiedKFold(n_splits = 5,
                        shuffle = True,
                        random_state = 24)

gcv = GridSearchCV(pipe, 
                  param_grid = params,
                  scoring = 'roc_auc',
                  cv = kfold)

gcv.fit(x, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('OHE',
                                        OneHotEncoder(handle_unknown='ignore',
                                                      sparse_output=False)),
                                       ('NB', GaussianNB())]),
             param_grid={'NB__var_smoothing': array([1.00000000e-03, 3.34222222e-01, 6.67444444e-01, 1.00066667e+00,
       1.33388889e+00, 1.66711111e+00, 2.00033333e+00, 2.33355556e+00,
       2.66677778e+00, 3.00000000e+00])},
             scoring='roc_auc')

In [112]:
gcv.best_score_

0.9565731081203064

In [ ]:
gcv